## Graph Tree Structure
#### Setting up necessary libraries

In [1]:
#Import ploty
import plotly 
import pandas as pd
import numpy as np
import re

### Input best_buy data
#### Put products_reduced_full_walkback.csv in your directory for input

In [2]:
bestbuy_pd = pd.read_csv("/Users/Maximus/Downloads/products_reduced_full_walkback.csv", delimiter = ",")
bestbuy_pd.head(3)

/Users/Maximus/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning:

Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.



,sku,productId,name,type,regularPrice,class,classId,subclass,subclassId,department,...,categoryParentLevel3Name,categoryParentLevel4Id,categoryParentLevel4Name,categoryParentLevel5Id,categoryParentLevel5Name,categoryParentLevel6Id,categoryParentLevel6Name,categoryParentLevel7Id,categoryParentLevel7Name,randId
0,3147106,139091,Wild as Springtime - CD,Music,14.99,COMPACT DISC,77,JAZZ-CONTEMPORARY,1002,VIDEO/COMPACT DISC,...,Best Buy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{54EB0588-A309-4FC9-80C8-00000758ED9F}
1,11157847,60139,Whity - Widescreen Subtitle - DVD,Movie,24.99,DVD SOFTWARE,126,DOCUMENTARIES,1029,VIDEO/COMPACT DISC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{E12DF265-D0EB-44D8-A7BE-00000879F0D6}
2,2619848,301613,Alone In His Presence - CASSETTE,Music,9.99,PRE-RECORDED CASS,80,GOSPEL,1055,VIDEO/COMPACT DISC,...,Best Buy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{B2C07C4E-1DA2-4973-BB9E-00000BF18EF0}


In [3]:
bestbuy_pd = bestbuy_pd.replace(np.nan,' ', regex=True)
bestbuy_pd.columns

Index([u'sku', u'productId', u'name', u'type', u'regularPrice', u'class',
       u'classId', u'subclass', u'subclassId', u'department',
       u'categoryPathId', u'categoryPathName', u'categoryParentLevel1Id',
       u'categoryParentLevel1Name', u'categoryParentLevel2Id',
       u'categoryParentLevel2Name', u'categoryParentLevel3Id',
       u'categoryParentLevel3Name', u'categoryParentLevel4Id',
       u'categoryParentLevel4Name', u'categoryParentLevel5Id',
       u'categoryParentLevel5Name', u'categoryParentLevel6Id',
       u'categoryParentLevel6Name', u'categoryParentLevel7Id',
       u'categoryParentLevel7Name', u'randId'],
      dtype='object')

## Interactive Dial button for user. (to be complete/design).
### This part fo the code is reserved for an interactive interface for users to input which columns are part of the data structure and needs to be udpated.

In [ ]:
### User informs: root node, columns names to use for 
##hireachical structure, and columns names use for feature engineering.

In [4]:
#In a fully set up site, this information will be obtained through user input.
#In order to make the process work, manually make the update for now.

hierarchy_col = ['categoryParentLevel7Name', 'categoryParentLevel6Name','categoryParentLevel5Name',
                 'categoryParentLevel4Name','categoryParentLevel3Name','categoryParentLevel2Name',
                 'categoryParentLevel1Name']
n_levels = len(hierarchy_col)
#root_node = ['categoryParentLevel7Name']

### The BestBuy dataset is not perfectly alinged column-wise so this type is a manual type to understand the structure of our data.  This part shouldn't neccessary for our product code when we assume users will update properly formatted data.

In [5]:
bestbuy_pd['combined'] = ''

for i in hierarchy_col:
  bestbuy_pd['combined'] = bestbuy_pd['combined'].astype(str) +'**'+ bestbuy_pd[i].astype(str)

bestbuy_pd['combined'] = bestbuy_pd['combined'].apply(lambda x: re.sub(r'.*Best Buy', 'Best Buy', x))
bestbuy_pd['combined'].head()

0          Best Buy**Movies & Music**Music
1                 Best Buy**Movies & Music
2          Best Buy**Movies & Music**Music
3          Best Buy**Movies & Music**Music
4    Best Buy**Movies & Music**Music**Rock
Name: combined, dtype: object

### Creating final dataset

In [80]:
New_Col = ['name', 'type', 'regularPrice', 'class', 'subclass', 'department']
new_data = pd.DataFrame(columns=New_Col)
new_data['name'] = bestbuy_pd['name']
new_data['type'] = bestbuy_pd['type']
new_data['regularPrice'] = bestbuy_pd['regularPrice']
new_data['class'] = bestbuy_pd['class']
new_data['subclass'] = bestbuy_pd['subclass']
new_data['department'] = bestbuy_pd['department']

In [29]:
foo = lambda x: pd.Series([i for i in x.split('**')])
levels = bestbuy_pd['combined'].apply(foo)

In [42]:
error_check = lambda x: pd.Series(len(x.split('-')))
test = bestbuy_pd['combined'].apply(error_check)

In [81]:
new_data = new_data.merge(levels, left_index=True, right_index = True)
new_data = new_data.merge(test, left_index=True, right_index = True)
#new_data.fillna(0, inplace = True)
new_data.columns = ['name', 'type', 'regularPrice', 'class', 'subclass', 'department', \
                    'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'level_6', 'level_7',\
                    'level_8', 'error_check']

In [82]:
new_data.loc[new_data['error_check']<2,\
['name', 'type', 'regularPrice', 'class', 'subclass', 'department', \
'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'level_6', 'level_7']].to_pickle('New_BestBuy_Data0402.p')

In [87]:
new_data.head()

,name,type,regularPrice,class,subclass,department,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,error_check
0,Wild as Springtime - CD,Music,14.99,COMPACT DISC,JAZZ-CONTEMPORARY,VIDEO/COMPACT DISC,Best Buy,Movies & Music,Music,NaN,NaN,NaN,NaN,NaN,1
1,Whity - Widescreen Subtitle - DVD,Movie,24.99,DVD SOFTWARE,DOCUMENTARIES,VIDEO/COMPACT DISC,Best Buy,Movies & Music,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Alone In His Presence - CASSETTE,Music,9.99,PRE-RECORDED CASS,GOSPEL,VIDEO/COMPACT DISC,Best Buy,Movies & Music,Music,NaN,NaN,NaN,NaN,NaN,1
3,Meal Ticket - CD,Music,12.99,COMPACT DISC,R&B,VIDEO/COMPACT DISC,Best Buy,Movies & Music,Music,NaN,NaN,NaN,NaN,NaN,1
4,Of the Son and the Father - CD,Music,13.99,COMPACT DISC,ROCK,VIDEO/COMPACT DISC,Best Buy,Movies & Music,Music,Rock,NaN,NaN,NaN,NaN,1


In [77]:
bestbuy_pd['combined'].head(10)

0          Best Buy**Movies & Music**Music
1                 Best Buy**Movies & Music
2          Best Buy**Movies & Music**Music
3          Best Buy**Movies & Music**Music
4    Best Buy**Movies & Music**Music**Rock
5          Best Buy**Movies & Music**Music
6          Best Buy**Movies & Music**Music
7          Best Buy**Movies & Music**Music
8    Best Buy**Movies & Music**Music**Rock
9                 Best Buy**Movies & Music
Name: combined, dtype: object

In [20]:
#Trying out Panda command
a = bestbuy_pd['combined'][0].split("**")
a= a + ['']*0
a
list(bestbuy_pd.loc[0, ['combined', 'sku']])

['Best Buy', 'Movies & Music', 'Music']